In [1]:
import tensorflow as tf
print('TensorFlow Version: {}'.format(tf.__version__))
print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.7.1
Default GPU Device: 


##### >**Note:** Make sure you have OpenAI Gym cloned. Then run this command `pip install -e gym/[all]`.

In [2]:
import numpy as np
import gym
env = gym.make('CartPole-v0')
env = gym.make('CartPole-v1')

In [3]:
def model_input(state_size):
    #states = tf.placeholder(tf.float32, [None, *state_size], name='states')
    states = tf.placeholder(tf.float32, [None, state_size], name='states')
    actions = tf.placeholder(tf.int32, [None], name='actions')
    next_states = tf.placeholder(tf.float32, [None, state_size], name='next_states')
    rewards = tf.placeholder(tf.float32, [None], name='rewards')
    dones = tf.placeholder(tf.float32, [None], name='dones')
    rates = tf.placeholder(tf.float32, [None], name='rates') # success rate
    return states, actions, next_states, rewards, dones, rates

In [4]:
def Act(states, action_size, hidden_size, reuse=False, alpha=0.1, training=False):
    with tf.variable_scope('Act', reuse=reuse):
        # First fully connected layer
        h1 = tf.layers.dense(inputs=states, units=hidden_size)
        bn1 = tf.layers.batch_normalization(h1, training=training)        
        nl1 = tf.maximum(alpha * bn1, bn1)
        
        # Second fully connected layer
        h2 = tf.layers.dense(inputs=nl1, units=hidden_size)
        bn2 = tf.layers.batch_normalization(h2, training=training)        
        nl2 = tf.maximum(alpha * bn2, bn2)
        
        # Output layer
        logits = tf.layers.dense(inputs=nl2, units=action_size)        
        return logits

In [5]:
def Env(states, actions, state_size, action_size, hidden_size, reuse=False, alpha=0.1, training=False):
    with tf.variable_scope('Env', reuse=reuse):
        # First fully connected layer
        h1 = tf.layers.dense(inputs=states, units=action_size)
        bn1 = tf.layers.batch_normalization(h1, training=training)        
        nl1 = tf.maximum(alpha * bn1, bn1)
        
        # Second fully connected layer
        nl1_fused = tf.concat(axis=1, values=[nl1, actions])
        h2 = tf.layers.dense(inputs=nl1_fused, units=hidden_size)
        bn2 = tf.layers.batch_normalization(h2, training=training)        
        nl2 = tf.maximum(alpha * bn2, bn2)
                
        # Output layer
        states_logits = tf.layers.dense(inputs=nl2, units=state_size, trainable=False)
        Qlogits = tf.layers.dense(inputs=nl2, units=1, trainable=False)
        return states_logits, Qlogits

In [6]:
def model_loss(state_size, action_size, hidden_size, gamma,
               states, actions, next_states, rewards, dones, rates):
    ################################################ a = act(s)
    actions_logits = Act(states=states, hidden_size=hidden_size, action_size=action_size)
    actions_labels = tf.one_hot(indices=actions, depth=action_size, dtype=actions_logits.dtype)
    aloss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=actions_logits, 
                                                                      labels=actions_labels))
    ################################################ s', r = env(s, a)
    next_states_logits, Qlogits = Env(actions=actions_labels, states=states, hidden_size=hidden_size, 
                                      action_size=action_size, state_size=state_size)
    next_states_logits_, Qlogits_ = Env(actions=actions_logits, states=states, hidden_size=hidden_size, 
                                        action_size=action_size, state_size=state_size, reuse=True)
    next_states_labels = tf.nn.sigmoid(next_states)
    eloss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=next_states_logits, # GQN real
                                                                   labels=next_states_labels))
    eloss += -tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=next_states_logits_, 
                                                                     labels=next_states_labels))
    aloss2 = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=next_states_logits_, 
                                                                    labels=next_states_labels))
    #################################################### Q=R MonteCarlo Prob-based
    Qlogits = tf.reshape(Qlogits, shape=[-1])
    Qlogits_ = tf.reshape(Qlogits_, shape=[-1])
    eloss += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=Qlogits, # GAN real
                                                                    labels=rates))
    eloss += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=Qlogits_, # min
                                                                    labels=tf.zeros_like(rates)))
    aloss2 += tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=Qlogits_,
                                                                     labels=tf.ones_like(rates)))
    #################################################### s'', Q' = ~env(s', ~a')
    next_actions_logits = Act(states=next_states, hidden_size=hidden_size, action_size=action_size, reuse=True)
    _next_states_logits_, nextQlogits_ = Env(actions=next_actions_logits, states=next_states, 
                                             hidden_size=hidden_size, action_size=action_size, 
                                             state_size=state_size, reuse=True)
    ###################################################### Q(s,a)= r + Q'(s',a')-TempDiff-Value-based
    nextQlogits_ = tf.reshape(nextQlogits_, shape=[-1]) * (1-dones)
    targetQs = rewards + (gamma * nextQlogits_)
    eloss += tf.reduce_mean(tf.square(Qlogits - targetQs)) # real
    eloss += tf.reduce_mean(tf.square(Qlogits_ - rewards)) # min
    #aloss2 += -tf.reduce_mean(tf.square(Qlogits_ - targetQs))
    aloss2 += -tf.reduce_mean(Qlogits_)
    return actions_logits, aloss, eloss, aloss2

In [7]:
def model_opt(a_loss, e_loss, a_loss2, a_learning_rate, e_learning_rate):
    # Get weights and bias to update
    t_vars = tf.trainable_variables()
    a_vars = [var for var in t_vars if var.name.startswith('Act')]
    e_vars = [var for var in t_vars if var.name.startswith('Env')]

    # Optimize
    with tf.control_dependencies(tf.get_collection(tf.GraphKeys.UPDATE_OPS)): # Required for batchnorm (BN)
        a_opt = tf.train.AdamOptimizer(a_learning_rate).minimize(a_loss, var_list=a_vars)
        e_opt = tf.train.AdamOptimizer(e_learning_rate).minimize(e_loss, var_list=e_vars)
        a_opt2 = tf.train.AdamOptimizer(a_learning_rate).minimize(a_loss2, var_list=a_vars)
    return a_opt, e_opt, a_opt2

In [8]:
class Model:
    def __init__(self, state_size, action_size, hidden_size, a_learning_rate, e_learning_rate, gamma):

        # Data of the Model: make the data available inside the framework
        self.states, self.actions, self.next_states, self.rewards, self.dones, self.rates = model_input(
            state_size=state_size)

        # Create the Model: calculating the loss and forwad pass
        self.actions_logits, self.a_loss, self.e_loss, self.a_loss2 = model_loss(
            state_size=state_size, action_size=action_size, hidden_size=hidden_size, gamma=gamma, # model init
            states=self.states, actions=self.actions, next_states=self.next_states, 
            rewards=self.rewards, dones=self.dones, rates=self.rates) # model input
        
        # Update the model: backward pass and backprop
        self.a_opt, self.e_opt, self.a_opt2 = model_opt(a_loss=self.a_loss, 
                                                        e_loss=self.e_loss,
                                                        a_loss2=self.a_loss2, 
                                                        a_learning_rate=a_learning_rate,
                                                        e_learning_rate=e_learning_rate)

In [9]:
from collections import deque
class Memory():
    def __init__(self, max_size = 1000):
        self.buffer = deque(maxlen=max_size) # data batch
#     def sample(self, batch_size):
#         idx = np.random.choice(np.arange(len(self.buffer)), size=batch_size, replace=False)
#         return [self.buffer[ii] for ii in idx]

## Hyperparameters

One of the more difficult aspects of reinforcememt learning are the large number of hyperparameters. Not only are we tuning the network, but we're tuning the simulation.

In [10]:
env.observation_space, env.action_space

(Box(4,), Discrete(2))

In [11]:
# Exploration parameters
explore_start = 1.0            # exploration probability at start
explore_stop = 0.01           # minimum exploration probability 
decay_rate = 0.0001            # exponential decay rate for exploration prob

# Network parameters
state_size = 4
action_size = 2
hidden_size = 4*2             # number of units in each Q-network hidden layer
a_learning_rate = 1e-4         # Q-network learning rate
e_learning_rate = 1e-4         # Q-network learning rate

# Memory parameters
memory_size = int(1e5)            # memory capacity
batch_size = int(1e3)             # experience mini-batch size
gamma=0.99

In [12]:
# Reset/init the graph/session
graph = tf.reset_default_graph()

# Init the model
model = Model(action_size=action_size, state_size=state_size, hidden_size=hidden_size, gamma=gamma,
              a_learning_rate=a_learning_rate, e_learning_rate=e_learning_rate)

# Init the memory
memory = Memory(max_size=memory_size)

In [13]:
state = env.reset()
total_reward = 0
num_step = 0
for _ in range(memory_size):
    action = env.action_space.sample()
    next_state, reward, done, _ = env.step(action)
    rate = -1
    memory.buffer.append([state, action, next_state, reward, float(done), rate])
    num_step += 1 # memory incremented
    total_reward += reward
    state = next_state
    if done is True:
        #print(done)
        rate = total_reward/500
        for idx in range(num_step): # episode length
            if memory.buffer[-1-idx][-1] == -1:
                memory.buffer[-1-idx][-1] = rate
        state = env.reset()
        total_reward = 0 # reset
        num_step = 0 # reset

## Training the model

Below we'll train our agent. If you want to watch it train, uncomment the `env.render()` line. This is slow because it's rendering the frames slower than the network can train. But, it's cool to watch the agent get better at the game.

In [ ]:
# Save/load the model and save for plotting
saver = tf.train.Saver()
episode_rewards_list, rewards_list = [], []
aloss_list, eloss_list, aloss2_list = [], [], []

# TF session for training
with tf.Session(graph=graph) as sess:
    sess.run(tf.global_variables_initializer())
    #saver.restore(sess, 'checkpoints/model.ckpt')    
    #saver.restore(sess, tf.train.latest_checkpoint('checkpoints'))
    total_step = 0 # Explore or exploit parameter
    episode_reward = deque(maxlen=100) # 100 episodes for running average/running mean/window

    # Training episodes/epochs
    for ep in range(11111):
        aloss_batch, eloss_batch, aloss2_batch = [], [], []
        total_reward = 0
        state = env.reset()
        num_step = 0
        rate = -1

        # Training steps/batches
        while True:
            # Explore (env) or Exploit (model)
            total_step += 1
            explore_p = explore_stop + (explore_start - explore_stop) * np.exp(-decay_rate * total_step) 
            if explore_p > np.random.rand():
                action = env.action_space.sample()
            else:
                action_logits = sess.run(model.actions_logits, feed_dict={model.states: state.reshape([1, -1])})
                action = np.argmax(action_logits)
            next_state, reward, done, _ = env.step(action)
            memory.buffer.append([state, action, next_state, reward, float(done), rate])
            num_step += 1 # momory added
            total_reward += reward
            state = next_state
            
            # Training with the maxrated minibatch
            batch = memory.buffer
            #for idx in range(memory_size// batch_size):
            while True:
                idx = np.random.choice(np.arange(memory_size// batch_size))
                states = np.array([each[0] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                actions = np.array([each[1] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                next_states = np.array([each[2] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                rewards = np.array([each[3] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                dones = np.array([each[4] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                rates = np.array([each[5] for each in batch])[idx*batch_size:(idx+1)*batch_size]
                states = states[rates >= np.max(rates)]
                actions = actions[rates >= np.max(rates)]
                next_states = next_states[rates >= np.max(rates)]
                rewards = rewards[rates >= np.max(rates)]
                dones = dones[rates >= np.max(rates)]
                rates = rates[rates >= np.max(rates)]
                if np.count_nonzero(dones) > 0 and len(dones) > 1 and np.max(rates) > 0:
                    break
            aloss, _ = sess.run([model.a_loss, model.a_opt],
                                  feed_dict = {model.states: states, 
                                               model.actions: actions,
                                               model.next_states: next_states,
                                               model.rewards: rewards,
                                               model.dones: dones,
                                               model.rates: rates})
            eloss, _ = sess.run([model.e_loss, model.e_opt],
                                  feed_dict = {model.states: states, 
                                               model.actions: actions,
                                               model.next_states: next_states,
                                               model.rewards: rewards,
                                               model.dones: dones,
                                               model.rates: rates})
            aloss2, _= sess.run([model.a_loss2, model.a_opt2], 
                                 feed_dict = {model.states: states, 
                                              model.actions: actions,
                                              model.next_states: next_states,
                                              model.rewards: rewards,
                                              model.dones: dones,
                                              model.rates: rates})
            # print(len(dones), np.count_nonzero(dones), np.max(rates))
            aloss_batch.append(aloss)
            eloss_batch.append(eloss)
            aloss2_batch.append(aloss2)
            if done is True:
                break
                
        # Rating the latest played episode
        rate = total_reward/500 # update rate at the end/ when episode is done
        for idx in range(num_step): # episode length
            if memory.buffer[-1-idx][-1] == -1: # double-check the landmark/marked indexes
                memory.buffer[-1-idx][-1] = rate # rate the trajectory/data

        # Print out
        episode_reward.append(total_reward)
        print('Episode:{}'.format(ep),
              'meanR:{:.4f}'.format(np.mean(episode_reward)),
              'R:{:.4f}'.format(total_reward),
              'rate:{:.4f}'.format(rate),
              'aloss:{:.4f}'.format(np.mean(aloss_batch)),
              'eloss:{:.4f}'.format(np.mean(eloss_batch)),
              'aloss2:{:.4f}'.format(np.mean(aloss2_batch)),
              'exploreP:{:.4f}'.format(explore_p))

        # Ploting out
        episode_rewards_list.append([ep, np.mean(episode_reward)])
        rewards_list.append([ep, total_reward])
        aloss_list.append([ep, np.mean(aloss_batch)])
        eloss_list.append([ep, np.mean(eloss_batch)])
        aloss2_list.append([ep, np.mean(aloss2_batch)])
        
        # Break episode/epoch loop
        ## Option 1: Solve the First Version
        #The task is episodic, and in order to solve the environment, 
        #your agent must get an average score of +30 over 100 consecutive episodes.
        if np.mean(episode_reward) >= 500:
            break
            
    # At the end of all training episodes/epochs
    saver.save(sess, 'checkpoints/model.ckpt')

Episode:0 meanR:15.0000 R:15.0000 rate:0.0300 aloss:0.6995 eloss:4.9318 aloss2:1.1188 exploreP:0.9985
Episode:1 meanR:18.0000 R:21.0000 rate:0.0420 aloss:0.6959 eloss:4.7816 aloss2:1.1598 exploreP:0.9964
Episode:2 meanR:17.6667 R:17.0000 rate:0.0340 aloss:0.6961 eloss:4.6653 aloss2:1.1863 exploreP:0.9948
Episode:3 meanR:18.0000 R:19.0000 rate:0.0380 aloss:0.6972 eloss:4.8367 aloss2:1.1046 exploreP:0.9929
Episode:4 meanR:16.8000 R:12.0000 rate:0.0240 aloss:0.6974 eloss:5.0496 aloss2:1.0221 exploreP:0.9917
Episode:5 meanR:17.3333 R:20.0000 rate:0.0400 aloss:0.6975 eloss:4.8413 aloss2:1.0824 exploreP:0.9898
Episode:6 meanR:19.2857 R:31.0000 rate:0.0620 aloss:0.6966 eloss:4.7396 aloss2:1.1033 exploreP:0.9867
Episode:7 meanR:18.8750 R:16.0000 rate:0.0320 aloss:0.6936 eloss:4.6206 aloss2:1.1190 exploreP:0.9852
Episode:8 meanR:18.0000 R:11.0000 rate:0.0220 aloss:0.6942 eloss:4.7261 aloss2:1.0811 exploreP:0.9841
Episode:9 meanR:18.8000 R:26.0000 rate:0.0520 aloss:0.6960 eloss:4.9926 aloss2:0.9

Episode:80 meanR:22.2346 R:34.0000 rate:0.0680 aloss:0.7257 eloss:3.2688 aloss2:1.0934 exploreP:0.8368
Episode:81 meanR:22.5122 R:45.0000 rate:0.0900 aloss:0.7186 eloss:3.2554 aloss2:1.0781 exploreP:0.8331
Episode:82 meanR:22.5422 R:25.0000 rate:0.0500 aloss:0.7288 eloss:3.2392 aloss2:1.0980 exploreP:0.8311
Episode:83 meanR:22.5238 R:21.0000 rate:0.0420 aloss:0.7246 eloss:3.2248 aloss2:1.0910 exploreP:0.8293
Episode:84 meanR:22.4588 R:17.0000 rate:0.0340 aloss:0.7320 eloss:3.2250 aloss2:1.0900 exploreP:0.8280
Episode:85 meanR:22.4186 R:19.0000 rate:0.0380 aloss:0.7314 eloss:3.2573 aloss2:1.0729 exploreP:0.8264
Episode:86 meanR:22.4253 R:23.0000 rate:0.0460 aloss:0.6979 eloss:3.1546 aloss2:1.1237 exploreP:0.8245
Episode:87 meanR:22.3636 R:17.0000 rate:0.0340 aloss:0.7207 eloss:3.2271 aloss2:1.0716 exploreP:0.8231
Episode:88 meanR:22.5843 R:42.0000 rate:0.0840 aloss:0.7181 eloss:3.1931 aloss2:1.0850 exploreP:0.8197
Episode:89 meanR:22.9111 R:52.0000 rate:0.1040 aloss:0.7136 eloss:3.1627 

Episode:159 meanR:24.6800 R:21.0000 rate:0.0420 aloss:0.7263 eloss:2.8453 aloss2:0.9744 exploreP:0.6866
Episode:160 meanR:25.1200 R:61.0000 rate:0.1220 aloss:0.7099 eloss:2.8323 aloss2:0.9818 exploreP:0.6825
Episode:161 meanR:25.1300 R:15.0000 rate:0.0300 aloss:0.7095 eloss:2.8282 aloss2:0.9657 exploreP:0.6815
Episode:162 meanR:25.6300 R:72.0000 rate:0.1440 aloss:0.7085 eloss:2.8175 aloss2:0.9596 exploreP:0.6767
Episode:163 meanR:25.7000 R:31.0000 rate:0.0620 aloss:0.6786 eloss:2.8193 aloss2:0.9512 exploreP:0.6746
Episode:164 meanR:26.1800 R:60.0000 rate:0.1200 aloss:0.6943 eloss:2.8157 aloss2:0.9378 exploreP:0.6706
Episode:165 meanR:26.4000 R:50.0000 rate:0.1000 aloss:0.6834 eloss:2.8086 aloss2:0.9288 exploreP:0.6673
Episode:166 meanR:26.3300 R:24.0000 rate:0.0480 aloss:0.7175 eloss:2.8098 aloss2:0.9127 exploreP:0.6658
Episode:167 meanR:26.7400 R:55.0000 rate:0.1100 aloss:0.6985 eloss:2.8088 aloss2:0.9082 exploreP:0.6622
Episode:168 meanR:26.9600 R:35.0000 rate:0.0700 aloss:0.6938 elo

Episode:238 meanR:49.6000 R:48.0000 rate:0.0960 aloss:0.7128 eloss:2.9814 aloss2:0.7694 exploreP:0.4434
Episode:239 meanR:50.3500 R:92.0000 rate:0.1840 aloss:0.7350 eloss:2.9744 aloss2:0.7655 exploreP:0.4394
Episode:240 meanR:51.0800 R:91.0000 rate:0.1820 aloss:0.7223 eloss:2.9823 aloss2:0.7519 exploreP:0.4356
Episode:241 meanR:52.7800 R:224.0000 rate:0.4480 aloss:0.7215 eloss:2.9807 aloss2:0.7487 exploreP:0.4261
Episode:242 meanR:54.3500 R:179.0000 rate:0.3580 aloss:0.7487 eloss:2.9796 aloss2:0.7455 exploreP:0.4188
Episode:243 meanR:54.8000 R:68.0000 rate:0.1360 aloss:0.7344 eloss:2.9732 aloss2:0.7466 exploreP:0.4160
Episode:244 meanR:55.1600 R:56.0000 rate:0.1120 aloss:0.7624 eloss:2.9905 aloss2:0.7341 exploreP:0.4137
Episode:245 meanR:55.2100 R:22.0000 rate:0.0440 aloss:0.7127 eloss:2.9548 aloss2:0.7560 exploreP:0.4128
Episode:246 meanR:55.9800 R:92.0000 rate:0.1840 aloss:0.7148 eloss:2.9922 aloss2:0.7360 exploreP:0.4091
Episode:247 meanR:56.8600 R:98.0000 rate:0.1960 aloss:0.7431 e

Episode:317 meanR:55.1100 R:25.0000 rate:0.0500 aloss:0.7499 eloss:3.0257 aloss2:0.7709 exploreP:0.2893
Episode:318 meanR:55.2400 R:34.0000 rate:0.0680 aloss:0.8144 eloss:3.0281 aloss2:0.7604 exploreP:0.2883
Episode:319 meanR:55.2200 R:48.0000 rate:0.0960 aloss:0.7489 eloss:3.0407 aloss2:0.7586 exploreP:0.2870
Episode:320 meanR:55.5300 R:44.0000 rate:0.0880 aloss:0.7382 eloss:3.0377 aloss2:0.7593 exploreP:0.2858
Episode:321 meanR:55.6600 R:35.0000 rate:0.0700 aloss:0.7585 eloss:3.0283 aloss2:0.7617 exploreP:0.2848
Episode:322 meanR:55.7000 R:40.0000 rate:0.0800 aloss:0.7514 eloss:3.0313 aloss2:0.7599 exploreP:0.2837
Episode:323 meanR:56.0000 R:46.0000 rate:0.0920 aloss:0.7548 eloss:3.0473 aloss2:0.7551 exploreP:0.2825
Episode:324 meanR:56.0400 R:42.0000 rate:0.0840 aloss:0.7456 eloss:3.0293 aloss2:0.7659 exploreP:0.2813
Episode:325 meanR:55.8300 R:46.0000 rate:0.0920 aloss:0.7331 eloss:3.0277 aloss2:0.7710 exploreP:0.2801
Episode:326 meanR:55.9600 R:51.0000 rate:0.1020 aloss:0.7576 elo

Episode:396 meanR:41.9700 R:37.0000 rate:0.0740 aloss:0.7870 eloss:3.0053 aloss2:0.7824 exploreP:0.2123
Episode:397 meanR:41.8500 R:41.0000 rate:0.0820 aloss:0.7541 eloss:2.9826 aloss2:0.7947 exploreP:0.2114
Episode:398 meanR:41.6400 R:35.0000 rate:0.0700 aloss:0.7616 eloss:3.0052 aloss2:0.7892 exploreP:0.2107
Episode:399 meanR:41.4700 R:44.0000 rate:0.0880 aloss:0.7723 eloss:2.9821 aloss2:0.8074 exploreP:0.2099
Episode:400 meanR:41.6000 R:49.0000 rate:0.0980 aloss:0.7330 eloss:2.9965 aloss2:0.7938 exploreP:0.2089
Episode:401 meanR:41.5300 R:36.0000 rate:0.0720 aloss:0.7634 eloss:3.0299 aloss2:0.7767 exploreP:0.2082
Episode:402 meanR:41.6600 R:64.0000 rate:0.1280 aloss:0.7588 eloss:3.0005 aloss2:0.7859 exploreP:0.2069
Episode:403 meanR:42.0500 R:80.0000 rate:0.1600 aloss:0.8091 eloss:2.9718 aloss2:0.8027 exploreP:0.2053
Episode:404 meanR:41.9800 R:43.0000 rate:0.0860 aloss:0.7686 eloss:2.9835 aloss2:0.8140 exploreP:0.2045
Episode:405 meanR:41.7600 R:31.0000 rate:0.0620 aloss:0.7574 elo

Episode:475 meanR:69.3100 R:114.0000 rate:0.2280 aloss:0.7302 eloss:3.0010 aloss2:0.8025 exploreP:0.1193
Episode:476 meanR:69.8800 R:91.0000 rate:0.1820 aloss:0.7391 eloss:3.0000 aloss2:0.8011 exploreP:0.1183
Episode:477 meanR:70.5000 R:96.0000 rate:0.1920 aloss:0.7350 eloss:2.9826 aloss2:0.8118 exploreP:0.1173
Episode:478 meanR:71.0500 R:91.0000 rate:0.1820 aloss:0.7431 eloss:2.9764 aloss2:0.8235 exploreP:0.1163
Episode:479 meanR:71.8200 R:117.0000 rate:0.2340 aloss:0.7359 eloss:2.9752 aloss2:0.8221 exploreP:0.1151
Episode:480 meanR:72.4400 R:94.0000 rate:0.1880 aloss:0.7383 eloss:2.9876 aloss2:0.8180 exploreP:0.1141
Episode:481 meanR:72.9000 R:82.0000 rate:0.1640 aloss:0.7372 eloss:2.9718 aloss2:0.8250 exploreP:0.1133
Episode:482 meanR:73.4400 R:81.0000 rate:0.1620 aloss:0.7546 eloss:2.9826 aloss2:0.7981 exploreP:0.1124
Episode:483 meanR:73.8000 R:81.0000 rate:0.1620 aloss:0.7492 eloss:2.9952 aloss2:0.8029 exploreP:0.1116
Episode:484 meanR:74.5900 R:108.0000 rate:0.2160 aloss:0.7324 

# Visualizing training

Below I'll plot the total rewards for each episode. I'm plotting the rolling average too, in blue.

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

def running_mean(x, N):
    cumsum = np.cumsum(np.insert(x, 0, 0)) 
    return (cumsum[N:] - cumsum[:-N]) / N 

In [ ]:
eps, arr = np.array(episode_rewards_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Total rewards')

In [ ]:
eps, arr = np.array(rewards_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Total rewards')

In [ ]:
eps, arr = np.array(aloss_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Act losses')

In [ ]:
eps, arr = np.array(eloss_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Env losses')

In [ ]:
eps, arr = np.array(aloss2_list).T
smoothed_arr = running_mean(arr, 10)
plt.plot(eps[-len(smoothed_arr):], smoothed_arr)
plt.plot(eps, arr, color='grey', alpha=0.3)
plt.xlabel('Episode')
plt.ylabel('Act losses 2')

## Testing

Let's checkout how our trained agent plays the game.

In [21]:
import gym
env = gym.make('CartPole-v0')
env = gym.make('CartPole-v1')

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    saver.restore(sess, 'checkpoints/model.ckpt')    
    #saver.restore(sess, tf.train.latest_checkpoint('checkpoints'))
    
    # Episodes/epochs
    for _ in range(10):
        state = env.reset()
        total_reward = 0

        # Steps/batches
        while True:
            env.render()
            action_logits = sess.run(model.actions_logits, feed_dict={model.states: state.reshape([1, -1])})
            action = np.argmax(action_logits)
            state, reward, done, _ = env.step(action)
            total_reward += reward
            if done:
                print('total_reward: {}'.format(total_reward))
                break
                
env.close()

INFO:tensorflow:Restoring parameters from checkpoints/model.ckpt
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
total_reward: 500.0
